In [ ]:
from dotenv import load_dotenv
from graph import create_graph_rag_variant
from langchain_core.runnables.graph import node_data_str

In [ ]:
load_dotenv()
app = create_graph_rag_variant()
app.get_graph().print_ascii()


In [ ]:
app.get_graph().nodes

In [ ]:
node1 = app.get_graph().nodes[1]
node1

In [ ]:
for node_name, node_info in app.get_graph().nodes.items():
    print(node_name)
    print(node_info)
    print()

In [ ]:
app.get_graph().edges

In [ ]:
nodes = {node.id: node_data_str(node) for node in app.get_graph().nodes}

In [1]:
from graph_state import GraphState
from langgraph.graph import END, StateGraph, MessageGraph
from agents import (
    check_for_hallucinations_and_relevance,
    decide_to_generate,
    generate,
    grade_documents,
    retrieve,
    route_query,
    web_search,
)


/home/arko/anaconda3/envs/rag-variants/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# wf = StateGraph(GraphState)
wf = MessageGraph()

# Define nodes
wf.add_node("websearch", web_search)
# wf.add_node("retrieve", retrieve)
# wf.add_node("grade_documents", grade_documents)
# wf.add_node("generate", generate)

# Define edges
wf.add_edge("websearch", END)

# Define entry point
wf.set_entry_point("websearch")

# Compile the graph
app = wf.compile()

# print the graph
app.get_graph().print_ascii()


+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+-----------+  
| websearch |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [4]:
app.get_graph()

Graph(nodes={'__start__': Node(id='__start__', data=<class 'pydantic.v1.main.LangGraphInput'>), '__end__': Node(id='__end__', data=<class 'pydantic.v1.main.LangGraphOutput'>), 'websearch': Node(id='websearch', data=RunnableLambda(web_search))}, edges=[Edge(source='__start__', target='websearch', data=None, conditional=False), Edge(source='websearch', target='__end__', data=None, conditional=False)])

In [14]:
import json

def graph_to_json(graph, output_file='rag_test.json'):
    # Extract nodes and edges from the graph
    nodes = [{'id': node.id, 'type': 'ellipse', 'data': node_data_str(node)} for node in graph.nodes]
    print(nodes)
    edges = [{'id': f"{edge.source.id}-{edge.target.id}", 'type': 'arrow', 'source': edge.source.id, 'target': edge.target.id} for edge in graph.get_edges()]
    
    # Prepare the JSON structure
    graph_json = {
        "elements": nodes + edges
    }
    
    # Write to JSON file
    with open(output_file, 'w') as json_file:
        json.dump(graph_json, json_file, indent=4)

    print(f"Graph data saved to {output_file}")

In [15]:
app.get_graph().nodes

{'__start__': Node(id='__start__', data=<class 'pydantic.v1.main.LangGraphInput'>),
 '__end__': Node(id='__end__', data=<class 'pydantic.v1.main.LangGraphOutput'>),
 'websearch': Node(id='websearch', data=RunnableLambda(web_search))}

In [16]:
graph_to_json(app.get_graph())

AttributeError: 'str' object has no attribute 'id'